In [1]:
from pathlib import Path

import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt

from dataloader import build_dataloader
from models import YOLOv3_Model
from utils import visualize

In [15]:
ROOT = Path.cwd()

EXP_NAME = 'test04'
data_path = ROOT / 'data' / 'coco128.yml'
config_path = ROOT / 'config' / 'yolov3.yml'
save_path = ROOT / 'experiments' / EXP_NAME

In [16]:
with open(config_path) as f:
    item = yaml.load(f, Loader=yaml.FullLoader)

In [17]:
log_level = item['LOG_LEVEL']
is_cuda = False
num_epochs = item['NUM_EPOCHS']
input_size = item['INPUT_SIZE']
batch_size = 1

device = torch.device('cuda' if torch.cuda.is_available() and is_cuda else 'cpu')
dataloaders, classname_list = build_dataloader(data_path=data_path, 
                                               image_size=(input_size, input_size),
                                               batch_size=batch_size)
num_classes = len(classname_list)
model = YOLOv3_Model(config_path=config_path, num_classes=num_classes, device=device)
model = model.to(device)

[TRAIN] hash: 22207404  version: 2022-06-30_22:28  
[VAL] hash: 7499800  version: 2022-07-04_21:13  


In [18]:
ckpt_path = save_path / 'weights' / '90.pth'
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt, strict=True)
model.eval()

YOLOv3_Model(
  (backbone): Darknet53_backbone(
    (conv1): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (lrelu): LeakyReLU(negative_slope=0.1)
    )
    (res_block1): Sequential(
      (conv): ConvLayer(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (lrelu): LeakyReLU(negative_slope=0.1)
      )
      (res0): ResBlock(
        (conv1): ConvLayer(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (lrelu): LeakyReLU(negative_slope=0.1)
        )
        (conv2): ConvLayer(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [19]:
def denormalize(input_tensor, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    tensor = input_tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    tensor.clamp_(min=0, max=1.)
    tensor *= 255.
    image = tensor.permute(1,2,0).numpy().astype(np.uint8)
    return image

def filter_conf_score(prediction, conf_threshold=0.01):
    valid_score_index = (prediction[..., 4] > conf_threshold)
    bboxes = prediction[:, :4][valid_score_index]
    conf_scores = prediction[:, 4][valid_score_index]
    class_probs = np.argmax(prediction[:, 5:][valid_score_index], axis=1)
    return np.concatenate([bboxes, conf_scores[:, np.newaxis], class_probs[:, np.newaxis]], axis=-1)

def run_NMS_yolo_format(bboxes, iou_threshold=0.1):
    if len(bboxes) == 0:
        return []
    if bboxes.dtype.kind == "i":
        bboxes = bboxes.astype("float")
    pick = []
    x1 = bboxes[:, 0] - bboxes[:, 2]/2
    y1 = bboxes[:, 1] - bboxes[:, 3]/2
    x2 = bboxes[:, 0] + bboxes[:, 2]/2
    y2 = bboxes[:, 1] + bboxes[:, 3]/2
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > iou_threshold)[0])))
    return pick

In [32]:
phase = 'val'
conf_threshold = 0.5
for index, mini_batch in enumerate(dataloaders['train']):
    images = mini_batch[0].to(device)
    targets = mini_batch[1]
    filenames = mini_batch[2]
    
    with torch.no_grad():
        predictions = model(images)
   
    canvas_image = denormalize(images[0])
    prediction = torch.cat(predictions, dim=1)[0]
    prediction = prediction.cpu().numpy()
    
    prediction = filter_conf_score(prediction, conf_threshold=conf_threshold)
    
    if index == 0:
        break

In [33]:
prediction.shape

(10647, 6)

In [34]:
canvas = visualize(canvas_image, prediction[:,:4], prediction[:,5], classname_list)

plt.figure(figsize=(6,6))
plt.imshow(canvas)
plt.axis('off')
plt.show()

In [119]:
def denormalize(input_tensor, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    tensor = input_tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    tensor.clamp_(min=0, max=1.)
    tensor *= 255.
    image = tensor.permute(1,2,0).numpy().astype(np.uint8)
    return image

def filter_conf_score(prediction, conf_threshold=0.01):
    valid_score_index = (prediction[..., 4] > conf_threshold)
    bboxes = prediction[:, :4][valid_score_index]
    conf_scores = prediction[:, 4][valid_score_index]
    class_probs = np.argmax(prediction[:, 5:][valid_score_index], axis=1)
    return np.concatenate([bboxes, conf_scores[:, np.newaxis], class_probs[:, np.newaxis]], axis=-1)

def run_NMS_yolo_format(bboxes, iou_threshold=0.1):
    if len(bboxes) == 0:
        return []
    if bboxes.dtype.kind == "i":
        bboxes = bboxes.astype("float")
    pick = []
    x1 = bboxes[:, 0] - bboxes[:, 2]/2
    y1 = bboxes[:, 1] - bboxes[:, 3]/2
    x2 = bboxes[:, 0] + bboxes[:, 2]/2
    y2 = bboxes[:, 1] + bboxes[:, 3]/2
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > iou_threshold)[0])))
    return pick

def cxcywh_to_x1y1x2y2(bboxes, input_size):
    xy_min = bboxes[:, 0:2] - bboxes[:, 2:4] / 2
    xy_max = bboxes[:, 0:2] + bboxes[:, 2:4] / 2
    xy_min = xy_min.clip(min=0.)
    xy_max = xy_max.clip(max=input_size)
    return np.concatenate((xy_min, xy_max), axis=1).astype(int)

In [120]:
phase = 'val'
conf_threshold = 0.8
iou_threshold = 0.4

for index, mini_batch in enumerate(dataloaders["train"]):
    images = mini_batch[0].to(device)
    targets = mini_batch[1]
    filenames = mini_batch[2]
    
    with torch.no_grad():
        predictions = model(images)
        
    if phase == 'val':
        canvas_image = denormalize(images[0])
        prediction = torch.cat(predictions, dim=1)[0]
        prediction = prediction.cpu().numpy()
        
        pred_res = filter_conf_score(prediction, conf_threshold=conf_threshold)
        valid_index = run_NMS_yolo_format(bboxes=pred_res[:, :4], iou_threshold=iou_threshold)
        pred_res = pred_res[valid_index]
        
    if index == 0:
        break

In [123]:
pred_res[:,:4]

array([[375.6864624 , 389.98358154,  26.37264824, 475.08203125],
       [343.54598999, 390.0592041 ,  29.67083931, 417.99578857],
       [ 16.53494453, 404.89187622, 317.46508789, 378.96646118],
       [401.04684448, 400.53598022, 295.54498291, 382.1991272 ],
       [207.91131592, 402.05981445, 306.59588623, 300.99389648],
       [264.42459106, 237.19659424, 324.0635376 , 437.43823242],
       [ 52.74630737, 183.06071472, 292.76083374, 354.44784546],
       [370.27770996,  49.84379578, 474.03161621, 321.41976929]])

In [121]:
canvas = visualize(canvas_image, pred_res[:,:4], pred_res[:,5], classname_list)

plt.figure(figsize=(6,6))
plt.imshow(canvas)
plt.axis('off')
plt.show()